# Use ssh agent to hold your keys

## Overview:
- **Teaching:** 20 min
- **Exercises:** 0 min

**Questions**
- Can I manage the deetails of my remotes to store details of credentials on different systems?
- Can I work more conveniently with keys so I don't have to use my passphrase everytime I access a resource?

**Objectives**
- Know how to check whether ssh agent is running and start it if necessary
- Know how to add keys to the agent
- Understand which keys to add and when it might be appropriate to time limit the keys

### Information:  Using config

Remembering your credentials and which key you use for different services can become non-trivial if you work with many remote systems.  You can use the ssh config file, `~/.ssh/config` to specify features of your connection e.g. if you have different usernames on different systems:

```bash
Host service
  IdentityFile id_ed25519_service
  User userid_service
```

### Enabling the SSH Agent

So far we have just replaced the need to enter a password to access a remote host with the need to enter a key pair passphrase.  Because of this is may be tempting to leave the passphrase empty when creating your key so that we do not have to enter it every time we access a service which may be many times a day.  This is poor security practise and is  likely to be in breach of the acceptable use policies covering the services you are accessing.

It is also a completely unnecessary risk as you can enable an agent on your local system so that you only have to enter the passphrase *once* and after that you will be able to access the remote system without entering the passphrase.  Here we will demostrate how to use `ssh-agent` but:

* on Linux you might want to consider the [GNOME keyring](https://wiki.gnome.org/Projects/GnomeKeyring)
* if using `putty` you can use [pageant](https://www.ssh.com/ssh/putty/putty-manuals/0.68/Chapter9.html)
* if using MobaXterm you can use its internal [MobAgent](https://help.jasmin.ac.uk/article/4832-mobaxterm)

### Information: Start `ssh-agent`

Most modern Linux distributions (and macOS) should have ssh-agent running by default. If your system does not then you should find the instructions for enabling it in your distribution using Google.  Typically you can check this with:

```bash
echo $SSH_AGENT_PID

```

If the output is empty then it isn't running.  It can be launched with:

```bash
eval `ssh-agent`
Agent pid 123
```

and now you can confirm it is running with:

```bash
echo $SSH_AGENT_PID
123
```

### Adding your key to the agent

To add the private part of your key pair to the SSH Agent, use the `ssh-add` command (on your local machine).  

```bash
ssh-add
```

By default this will add the files: `~/.ssh/id_rsa`, `~/.ssh/id_dsa`, `~/.ssh/id_ecdsa`, `~/ssh/id_ed25519` and `~/.ssh/identity`, if they exist.

If we want to add a specific key that is not one of these we must specify it explicitly:

```bash
ssh-add ~/.ssh/id_ed25519-service
Enter passphrase for home/user/.ssh/id_ed25519_service: [Passphrase]
Identity added: home/user/.ssh/id_ed25519_service (home/user/.ssh/id_ed25519_service)
```

We can also add keys for a specific length of time.  To add the key for one hour we inculde the flag and parameter `-t 3600`, you will need to enter your passphrase one more time:

```bash
ssh-add -t 3600 ~/.ssh/id_ed25519-service
Enter passphrase for home/user/.ssh/id_ed25519_service: [Passphrase]
Identity added: home/user/.ssh/id_ed25519_service (home/user/.ssh/id_ed25519_service)
Lifetime set to 3600 seconds
```

Now you can test that you can access the remote host without needing to enter your passphrase:

```bash
ssh [userID]@<hpc-service> 'date'
Wed May  8 10:42:56 BST 2020
```

again we have run date on the remote service to confirm that we have been able to use the ssh-agent successfully.

Remember that in the above `user` will be your username on your local machine and that `[userID]` is you username on the remote `<hpc-service>`.

### Information: Agent lifetime

By default ssh-agent will store your key *forever*, until the machine is rebooted.  Remember that we are trying to ensure that we operate as securely as possible.  If we could be completely confident that our local machine could not be taken and compromised then we would not use passwords.

Similarly with the ssh-agent we must consider how long that the passphrase needs to be or should be remembered.  If we are in a secure office at work and we will be accessing the service repeatedly throughout the day then we might want the key to be remembered for several hours.  If we are doing half an hour's work in a cafe we would probably want to have the keys stored for that length of time.  Note that the time is in seconds.

At the end of a session you can remove all stored keys with:

```bash
ssh-add -D
```

### Information: Moving data

If you use `scp`, `rsync` or `sftp` to transfer files then since these use SSH they will use your ssh config file and stored keys in exactly the same way as running ssh.  

## Key Points:

- Check whether the ssh agent is running with `echo $SSH_AGENT_PID`
- If the output is empty then it isn't running. It can be launched with:
```
eval `ssh-agent`
```
- Add default key names with `ssh-add` or specific keys with `ssh-add ~/.ssh/id_ed25519-service`
- Think about whether it might be prudent to time limite your keys with `ssh-add -t XXXX`
- Remove all keys from the agent with `ssh-add -D`